# Claim & Image Peek

Interactively browse the AVeriMaTec validation claims and preview the images referenced in each record.

> Set the slider (or use the random button) below to inspect a claim, its metadata, the claim images, and question-level input images.

In [1]:
from pathlib import Path
import json
import random

from IPython.display import display, HTML
from PIL import Image
import ipywidgets as widgets

VAL_JSON_PATH = Path('/mnt/data/factcheck/averimatec/val.json')
IMAGES_DIR = Path('/mnt/data/factcheck/averimatec/images')

if not VAL_JSON_PATH.exists():
    raise FileNotFoundError(f'Missing val.json at {VAL_JSON_PATH}')
if not IMAGES_DIR.exists():
    raise FileNotFoundError(f'Missing images directory at {IMAGES_DIR}')

print(f'val.json located at: {VAL_JSON_PATH}')
print(f'images directory located at: {IMAGES_DIR}')

val.json located at: /mnt/data/factcheck/averimatec/val.json
images directory located at: /mnt/data/factcheck/averimatec/images


In [2]:
with VAL_JSON_PATH.open('r', encoding='utf-8') as f:
    claims = json.load(f)

print(f'Loaded {len(claims):,} validation claims.')

Loaded 152 validation claims.


In [3]:
def _image_path(image_name: str) -> Path:
    return IMAGES_DIR / image_name


def display_image(image_name: str, caption: str = 'Image') -> None:
    path = _image_path(image_name)
    if not path.exists():
        display(HTML(f"<b>Missing image file:</b> {image_name}"))
        return

    with Image.open(path) as img:
        display(HTML(f"<em>{caption}</em> — {image_name} ({img.width}×{img.height}px)"))
        display(img)


def render_claim(idx: int) -> None:
    if not (0 <= idx < len(claims)):
        raise IndexError(f'Index {idx} is outside the dataset range (0-{len(claims) - 1}).')

    claim = claims[idx]
    metadata = claim.get('metadata', {})

    header_html = f"""
    <h3>Claim #{idx}</h3>
    <ul>
        <li><b>Date:</b> {claim.get('date', '—')}</li>
        <li><b>Label:</b> {claim.get('label', '—')}</li>
        <li><b>Article:</b> <a href='{claim.get('article', '')}' target='_blank'>{claim.get('article', '')}</a></li>
        <li><b>Claim text:</b> {claim.get('claim_text', '—')}</li>
        <li><b>Speaker:</b> {metadata.get('speaker', '—')}</li>
    </ul>
    <p><b>Justification:</b> {claim.get('justification', '—')}</p>
    """
    display(HTML(header_html))

    if claim.get('claim_images'):
        display(HTML('<h4>Claim images</h4>'))
        for image_name in claim['claim_images']:
            display_image(image_name, caption='Claim image')
    else:
        display(HTML('<p>No claim-level images listed.</p>'))

    if claim.get('questions'):
        display(HTML('<h4>Questions & evidence</h4>'))
        for q_idx, question in enumerate(claim['questions']):
            question_html = f"""
            <details open>
                <summary><b>Question {q_idx + 1}:</b> {question.get('question', '—')}</summary>
                <ul>
                    <li><b>Answer method:</b> {question.get('answer_method', '—')}</li>
                    <li><b>Question type:</b> {', '.join(question.get('question_type', [])) or '—'}</li>
                </ul>
            </details>
            """
            display(HTML(question_html))

            for ans in question.get('answers', []):
                answer_html = f"""
                <p><b>Answer ({ans.get('answer_type', '—')}):</b> {ans.get('answer_text', '—')}</p>
                <p><b>Source:</b> <a href='{ans.get('source_url', '')}' target='_blank'>{ans.get('source_url', '')}</a> ({ans.get('source_medium', '—')})</p>
                """
                display(HTML(answer_html))

            if question.get('input_images'):
                display(HTML('<p><b>Input images:</b></p>'))
                for image_name in question['input_images']:
                    display_image(image_name, caption='Question image')
            else:
                display(HTML('<p>No question-level images.</p>'))
    else:
        display(HTML('<p>No questions attached to this claim.</p>'))

In [4]:
index_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(claims) - 1,
    step=1,
    description='Claim #',
    continuous_update=False,
    readout=True,
    readout_format='d'
)

random_button = widgets.Button(description='Random claim', icon='shuffle')
output = widgets.Output()


def update_view(idx: int) -> None:
    output.clear_output()
    with output:
        render_claim(idx)


def on_slider_change(change):
    if change['name'] == 'value' and change['new'] is not None:
        update_view(change['new'])


def on_random_click(_):
    index_slider.value = random.randint(index_slider.min, index_slider.max)

index_slider.observe(on_slider_change, names='value')
random_button.on_click(on_random_click)

display(widgets.HBox([index_slider, random_button]))
display(output)

update_view(index_slider.value)

Output()